In [1]:
# Import the random module.
import random

In [6]:
random_number = random.randint(-90, 89) + random.random()
random_number

-86.09073985159036

In [7]:
x = 1
latitudes = []
while x < 11:
    random_lat = random.randint(-90, 89) + random.random()
    latitudes.append(random_lat)
    x += 1

In [8]:
# Import timeit.
import timeit

# Import the NumPy module.
import numpy as np

In [9]:
%timeit np.random.uniform(-90.000, 90.000, size=1500)

12 µs ± 499 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [10]:
def latitudes(size):
    latitudes = []
    x = 0
    while x < (size):
        random_lat = random.randint(-90, 90) + random.random()
        latitudes.append(random_lat)
        x += 1
    return latitudes
# Call the function with 1500.
%timeit latitudes(1500)

1.65 ms ± 35.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
